# The central limit theorem
## Understanding via visualization
#### Giovanni Pizzi (EPFL), Sep 2018
[Go back to the list of all visualizations](https://github.com/giovannipizzi/educational-scientific-visualizations/)

# Aim of this app
The aim of this app is to:
- visually prove the central limit theorem
- give a feeling on how fast the normal distribution is obtained when you sum random distributions

Read the _tasks_ below and play with the sliders to see the result!

# The central limit theorem
Suppose ${X_1, X_2, \ldots}$ is a sequence of _independent and identically distributed_ random variables with expectation value $\text{E}[X_i] = \mu$ and variance $\text{Var}[X_i] = \sigma^2$. Then, in the limit of $n\to\infty$, the random variables 
$$
S_N = \sum_{i=1}^N X_i
$$
converge in distribution to a normal distribution with the following expectation value and variance:
$$
\lim_{N\to\infty} \text{E}[S_N] = N\mu, \qquad \lim_{N\to\infty} \text{Var}[S_N] = N\sigma^2.
$$

# Tasks
- <span style="color: #cc0000">Move the sliders below and look at the results.</span>
- <span style="color: #cc0000">Verify that (for a uniform distribution) $N\geq 3$ already the distribution approximates quite accurately a random distribution!!</span>
- <span style="color: #cc0000">Check how fast (as a function of $N$) the distribution converges to a normal distribution with other distributions.</span>
- <span style="color: #cc0000">Check the importance of having a large number of $n_\text{samples}$ to achieve a good convergence.</span>


# Numerical verification and visualization
With the following selectors, you can pick:
- the number $N$ of random variables to add
- the number $n_\text{samples}$ of random numbers that you want to generate for every sequence
- the number $n_\text{bins}$ of histogram bins
- the type of random distribution

In [ ]:
import math
import numpy as np
import bqplot.pyplot as pl

In [ ]:
def get_numerical_arrays(number_of_samples, number_of_addends, bins, distrib_type):
    if distrib_type == "uniform":
        all_numbers = np.random.rand(number_of_addends, number_of_samples)
        
        description_string = (
            r"<strong>Distribution</strong>: random numbers in the [0,1[ range, uniformly distributed<br>"
            r"<strong>Distribution average</strong>: $\text{E}(X_i) = \int_0^1 x\, dx = \frac 1 2$<br>"
            r"<strong>Distribution variance</strong>: $\text{Var}(X_i) = \int_0^1 [x-\text{E}(X_i)]^2\, dx = \frac 1 {12}$<br>")
        average = 0.5 
        variance = 1./12.   
    elif distrib_type == "squared":    
        all_numbers = np.random.rand(number_of_addends, number_of_samples)**2
        
        description_string = (
            r"<strong>Distribution</strong>: random numbers in the [0,1[ range, uniformly distributed, then squared<br>"
            r"<strong>Distribution average</strong>: $\text{E}(X_i) = \int_0^1 x^2\, dx = \frac 1 3$<br>"
            r"<strong>Distribution variance</strong>: $\text{Var}(X_i) = \int_0^1 [x^2-\text{E}(X_i)]^2\, dx = \frac 4 {45}$<br>")

        average = 1./3.
        variance = 4./45.


    elif distrib_type == "squareroot":    
        all_numbers = np.sqrt(np.random.rand(number_of_addends, number_of_samples))
        
        description_string = (
            r"<strong>Distribution</strong>: random numbers in the [0,1[ range, uniformly distributed, then considering their square root<br>"
            r"<strong>Distribution average</strong>: $\text{E}(X_i) = \int_0^1 \sqrt{x}\, dx = \frac 3 2$<br>"
            r"<strong>Distribution variance</strong>: $\text{Var}(X_i) = \int_0^1 [\sqrt{x}-\text{E}(X_i)]^2\, dx = \frac 1 {18}$<br>")

        average = 2./3.
        variance = 1./18.

    else:
        raise NotImplementedError("Unknown type '{}'".format(distrib_type))
    
    all_numbers_sum = all_numbers.sum(axis=0) 
    y, x_edges = np.histogram(all_numbers_sum, bins=bins)
    x = (x_edges[1:] + x_edges[:-1])/2.

    mu = average * number_of_addends
    sigma = math.sqrt(variance * number_of_addends)
    bin_width = x_edges[1] - x_edges[0]
    norm = number_of_samples * bin_width

    gaussian_x = np.linspace(x[0], x[-1], 300)
    gaussian_y = 1./math.sqrt(2 * math.pi * sigma**2) * np.exp(-(gaussian_x-mu)**2/2/sigma**2) * norm
    return x, y, gaussian_x, gaussian_y, description_string

In [ ]:
from ipywidgets import Accordion, IntSlider, HTMLMath, Dropdown, Box, HBox, VBox, Layout
from IPython.display import display

n_widget = IntSlider(value=2, min=1, max=10, description = "$N$", continuous_update=False)
n_samples_widget = IntSlider(value=50000, min=100, max=100000, description = r"$n_{\text{samples}}$", continuous_update=False)
n_bins_widget = IntSlider(value=100, min=20, max=400, description = r"$n_{\text{bins}}$", continuous_update=False)
type_widget = Dropdown(options=(
        ("Uniform","uniform"),
        ("Squared","squared"),
        ("Square root","squareroot"),    
    ), 
    description = "Distrib. type", continuous_update=False, layout=Layout(width='250px'))

distribution_plot = pl.figure()
result_plot = pl.figure()
distribution_description = HTMLMath(value="")

distribution_accordion = Accordion(children=[distribution_plot], layout=Layout(width='90%', max_width='400px'))
distribution_accordion.set_title(0, 'Plot of a single distribution X')
# Start closed
distribution_accordion.selected_index = None

In [ ]:
def on_distrib_params_change(change):        
    x1, y1, _, _, _ = get_numerical_arrays(
        number_of_samples=n_samples_widget.value, 
        number_of_addends=1, # one single distribution
        bins=n_bins_widget.value,
        distrib_type=type_widget.value)

    x, y, gaussian_x, gaussian_y, description_string = get_numerical_arrays(
        number_of_samples=n_samples_widget.value, 
        number_of_addends=n_widget.value, 
        bins=n_bins_widget.value,
        distrib_type=type_widget.value)
        
    distribution_description.value = description_string
    
    pl.figure(fig=distribution_plot)
    pl.clear()
    pl.bar(x1,y1)
    pl.ylim(0,max(y1)*1.3)
    pl.xlabel("Value")
    pl.ylabel("Distribution of results")

    pl.figure(fig=result_plot)
    result_plot.legend_location = 'top-left'
    pl.clear()
    reverse_options_map = {_[1]: _[0] for _ in type_widget.options}
    pl.title('Distribution S (sum of {} "{}" distributions X)'.format(
        n_widget.value, 
        reverse_options_map[type_widget.value].lower()
    ))
    pl.bar(x,y, labels=["Distribution S"])
    pl.plot(gaussian_x,gaussian_y, labels=["Theoretical limit"], colors=["#ff0000"])
    pl.legend()
    pl.xlabel("Value")
    pl.ylabel("Distribution of results")

n_widget.observe(on_distrib_params_change, names='value', type='change')
n_samples_widget.observe(on_distrib_params_change, names='value', type='change')
n_bins_widget.observe(on_distrib_params_change, names='value', type='change')
type_widget.observe(on_distrib_params_change, names='value', type='change')
# Create the plot
on_distrib_params_change(None)

In [ ]:
display(Box([
        VBox([n_widget, n_samples_widget, n_bins_widget, type_widget], layout=Layout(width='350px')),
        VBox([distribution_description, distribution_accordion], layout=Layout(min_width='300px')),
    ], layout=Layout(width='100%', flex_flow='row wrap', display='flex')))

In [ ]:
result_plot.layout.width = '100%'
result_plot.layout.max_width = '800px'
distribution_plot.layout.width = '100%'
display(Box(children=[result_plot], layout=Layout(justify_content='center')))

# References
[1] [Central limit theorem on Wikipedia](https://en.wikipedia.org/wiki/Central_limit_theorem)

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))